In [ ]:
import requests
from bs4 import BeautifulSoup
import selenium

In [22]:
url = "https://gesetze.berlin.de/bsbe/document/jlr-SchulGBErahmen"
res = requests.get(url)
res.raise_for_status()
res.text

'<!doctype html>\n<html lang=\'de\'>\n<head>\n    <meta charset=\'utf-8\'>\n    <meta name=\'juris-version\' content=\'bsbe - V08_22_00 - 13.08.2025 11:23\'>\n    <meta name=\'viewport\' content=\'width=device-width, initial-scale=1, shrink-to-fit=no\'>\n    <meta name=\'theme-color\' content=\'#000000\'>\n    <meta name=\'tdm-reservation\' content=\'1\'>\n    <link rel=\'manifest\' href=\'/bsbe/manifest.json\'>\n    <link rel=\'shortcut icon\' href=\'/jportal/cms/technik/media/bs/berlin/faviconbsbe.ico\' />\n    <title>Berliner Vorschriften- und Rechtsprechungsdatenbank</title>\n    <meta http-equiv=\'x-ua-compatible\' content=\'ie=edge\' />\n    <meta name=\'author\' content=\'juris GmbH\' />\n    <meta name=\'keywords\' content=\'juristische Informationen, Rechtsinformationen\' />\n    <meta name=\'description\' content=\'Recherche juristischer Informationen\' />\n    <meta name=\'robots\' content=\'nofollow\' />\n    <meta name=\'format-detection\' content=\'telephone=no\' />\n    

In [23]:
soup = BeautifulSoup(res.content)

In [ ]:
soup.find(class_="docbody")

AttributeError: 'NoneType' object has no attribute 'text'

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import os


def scrape_law_page(url, version="standard", timeout=10):
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=chrome_options)

    driver.get(url)

    time.sleep(4)
    if version == "standard":
        button_xpath = (
            "//span[contains(normalize-space(), 'Aktueller Gesamttext') "
            "or contains(normalize-space(), 'Aktuelle Gesamtausgabe')]"
        )

        button = WebDriverWait(driver, timeout).until(
            EC.element_to_be_clickable((By.XPATH, button_xpath))
        )

        button.click()
        time.sleep(3)
    
        content_class = "docbody"

        page_source = driver.page_source

        driver.quit()

        soup = BeautifulSoup(page_source, "html.parser")

        doc_body = soup.find(class_=content_class)
    elif version == "wolterskluwer":
        button_xpath = "//a[contains(text(), 'Gesamte Quelle anzeigen')]"
        button = WebDriverWait(driver, timeout).until(
            EC.element_to_be_clickable((By.XPATH, button_xpath))
        )

        button.click()
        time.sleep(3)
        scroll_increment_ratio = 20
        pause_time = 0.1

        last_position = driver.execute_script("return window.pageYOffset")
        total_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollBy(0, arguments[0]);", total_height // scroll_increment_ratio)
            time.sleep(pause_time)

            current_position = driver.execute_script("return window.pageYOffset")

            total_height = driver.execute_script("return document.body.scrollHeight")

            viewport_height = driver.execute_script("return window.innerHeight")

            if current_position + viewport_height >= total_height:
                break

            if current_position == last_position:
                break

            last_position = current_position

        content_class = "block-system-main-block"

        page_source = driver.page_source

        soup = BeautifulSoup(page_source, "html.parser")

        doc_body = soup.find(class_=content_class)

    driver.quit()
    if doc_body:
        return doc_body.prettify()
    else:
        print("No element with class 'docbody' found.")
        return None


url = "https://recht.saarland.de/bssl/document/jlr-SchulOGSLrahmen"
content = scrape_law_page(url)
# content = scrape_law_page(url, version="wolterskluwer")
if content:
    print(content)


<article class="docbody docview__docbody" data-juris-toc='{"level":0,"label":"jlr-SchulOGSLrahmen"}'>
 <!--hlIgnoreOff-->
 <a data-juristest-docid="jlr-SchulOGSLrahmen" name="DocInhalt">
  <!--emptyTag-->
 </a>
 <!--slcase-->
 <div class="docLayoutTitel">
  <div class="doc" style="margin-left: -20px; margin-top: 0.5ex;">
   <h3>
    <span class="h3_titel">
     Gesetz Nr. 812 zur Ordnung des Schulwesens im Saarland
     <br/>
     (Schulordnungsgesetz - SchoG)
     <br/>
     Vom 5. Mai 1965
     <br/>
     in der Fassung der Bekanntmachung vom 21. August 1996
    </span>
   </h3>
   <!--emptyTag-->
  </div>
 </div>
 <div class="docLayoutText doktyp-juris-lr dokutyp-sl">
  <div class="docLayoutMarginTopMore">
   <em>
    Zum 20.08.2025 aktuellste verfügbare Fassung der Gesamtausgabe
   </em>
  </div>
  <div class="docLayoutMarginTop">
   <div class="docLayoutMarginTop">
    <a name="jlr-SchulOGSLrahmen">
     <!--emptyTag-->
    </a>
    <div class="docLayoutMarginTop">
     <div class

In [7]:
"Ersatzschulen dürfen nur mit vorheriger Genehmigung" in content

True

In [11]:
from httpx import head
import requests


def get_law_soup(url, class_):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
    }

    res = requests.get(url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.content)
    content = soup.find(class_=class_)

    if content:
        return str(content)
    else:
        raise RuntimeError("Did not find law content")


get_law_soup(
    "https://bass.schule.nrw/834.htm",
    "content-area",
)

'<div class="col content-area" id="menuheader">\n<div class="landesmaster">\n<section class="col background">\n<div class="dl-horizontal row">\n<div class="col col-md-3 col-sm-12 col-12 p-1 non-printable">\n<div class="content-area">\n<ul class="list-group list-unstyled">\n<li class="list-item"><a id="printPage"><i class="fas fa-print"></i>Drucken</a></li>\n<li class="list-item"><a href="/pdf/834.pdf?20250820174127" target="_blank"><i class="far fa-file-pdf"></i>als PDF speichern</a></li>\n<li class="list-item"><a href="/docx/834.docx?20250820174127" target="_blank"><i class="far fa-file-word"></i>als Word speichern</a></li>\n<li class="list-item"><a href="mailto:?subject=Link%20zum%2017-51%20Nr.%201%20Die%20Mitwirkung%20der%20Sch%C3%BClervertretung%20in%20der%20Schule%20%28SV-Erlass%29&amp;body=https://bass.schule.nrw/834.htm" target="_blank"><i class="fas fa-envelope-square"></i>als E-Mail verschicken</a></li>\n<li class="list-item"><hr/></li>\n<li class="list-item"><a class="faceboo

In [ ]:
def wait_jportal_load(url):
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=chrome_options)

    driver.get(url)
    time.sleep(4)
    page_source = driver.page_source
    driver.quit()
    
    soup = BeautifulSoup(page_source)

    doc_body = soup.find(class_="docbody")

    driver.quit()
    if doc_body:
        return doc_body.prettify()
    else:
        print("No element with class 'docbody' found.")
        return None
    

print(portal_load("https://landesrecht.rlp.de/bsrp/document/VVRP-VVRP000003549"))

<article class="docbody docview__docbody" data-juris-toc='{"level":0,"label":"VVRP-VVRP000003549"}'>
 <!--hlIgnoreOff-->
 <a data-juristest-docid="VVRP-VVRP000003549" name="DocInhalt">
  <!--emptyTag-->
 </a>
 <!--slcase-->
 <div class="docLayoutTitel">
  <div class="doc" style="margin-left: -20px; margin-top: 0.5ex;">
   Aufgaben, Wahl und Verfahrensweise der Vertretungen für Schülerinnen und Schüler
   <!--emptyTag-->
  </div>
 </div>
 <div class="docLayoutText doktyp-juris-v dokutyp-vv-rp-100-u">
  <div class="docLayoutMarginTop">
   <div>
    <h3 class="doc">
     Nichtamtliches Inhaltsverzeichnis
    </h3>
   </div>
   <a name="ivz">
    <!--emptyTag-->
   </a>
   <ul style="list-style-type:none;padding-top:0px;padding-bottom:0px;margin-top:0px;margin-bottom:0px;">
    <li style="text-decoration:none;margin-left=0px">
     <a class="doclink docview__doclink doclink--visited docview__doclink--visited" data-juris-gui="link" data-juris-link='{"linkMeta":{"docId":"VVRP-VVRP000003549",